In [ ]:
from glass.mob.esri import service_areas

facilities = '/mnt/disk1/dist_to_club/loc_clubs.shp'

breaks = ['5', '10', '15']

out = '/mnt/disk1/dist_to_club/sv_clubs_tv.shp'

impedance = 'TravelTime'

service_areas(facilities, breaks, out, impedance=impedance)

In [1]:
from glass.mob.esri.cf import closest_facility

i   = '/mnt/e/rms/madeira/rms_bgri11_pnt.shp'
ipk = 'bgri'

f = '/mnt/e/rms/equipasocial/creches_mad.shp'
out = '/mnt/e/rms/acc_pop21/acc_creches_psanto.shp'

imp = 'WalkTime'

epsg = 5016

closest_facility(
    i, ipk, f, out, impedance=imp,
    crs=epsg, save_temp_json=True
)

/home/jasp/.virtualenvs/genv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


TypeError: 'NoneType' object is not iterable

In [ ]:
import geopandas as gp
from glass.rd.shp import shp_to_obj

In [ ]:
i   = '/mnt/e/rms/azores/rms_bgri21_pnt_smiguel.shp'
ipk = 'bgri'

f = '/mnt/e/rms/equipasocial/idosos_saomiguel.shp'

In [ ]:
idf = shp_to_obj(i)
fdf = shp_to_obj(f)

#idf = idf[idf[ipk] == '42030700103']

In [ ]:
idf['auxcol'] = 1
fdf['auxcol'] = 1

idf['fid_a'] = idf.index + 1
fdf['fid_b'] = fdf.index + 1

adf = idf.merge(fdf, how='inner', on='auxcol').drop('auxcol', axis=1)

In [ ]:
geom_i = gp.GeoSeries(adf.geometry_x)
geom_f = gp.GeoSeries(adf.geometry_y)

adf["dist"] = geom_i.distance(geom_f, align=True) / 1000

In [ ]:
adf = adf[adf.dist < 20]
adf = adf[['fid_a', 'fid_b']]
# Get groups
adf = adf.groupby('fid_a')['fid_b'].apply(list).reset_index(name='fid_b')
adf['fid_b'] = adf.fid_b.apply(lambda x: sorted(x))
adf = adf.groupby(adf.fid_b.map(tuple))['fid_a'].apply(list).reset_index(name='fid_a')

In [ ]:
for i, group in adf.iterrows():
    inc = idf[idf['fid_a'].isin(group.fid_a)]
    fac = fdf[fdf['fid_b'].isin(group.fid_b)]

    fids = inc.fid_a.tolist()

    if 349 in fids:
        print(i)
        break

In [ ]:
idfs[2]

In [ ]:
from glass.pd.split import df_split

idfs = df_split(inc, 100, nrows=True)

In [ ]:
import json
from glass.it.esri import json_to_gjson
from glass.it.pd      import json_obj_to_geodf

js = '/mnt/e/rms/acc_pop21/esri_response_4_5.json'

esri_geom = json.load(open(js, 'r'))
geom = json_to_gjson(esri_geom.get('routes'))

gdf = json_obj_to_geodf(geom, 4326)

In [ ]:
gdf.shape

In [ ]:
from glass.mob.esri.cf import cf_ign_distfac

i   = '/mnt/e/rms/azores/rms_bgri11_pnt.shp'
ipk = 'bgri'

f = '/mnt/e/rms/equipasocial/creches_azores.shp'
out = '/mnt/e/rms/acc_pop21/acc_creches_azores.shp'

epsg = 5015

cf_ign_distfac(
    i, ipk, f, out,
    crs=epsg, save_temp_json=True
)

-----------------------------------

### Loop of cf_ign_distfac

In [ ]:
import os
from glass.pys.oss import lst_fld
from glass.mob.esri.cf import cf_ign_distfac

In [ ]:
folder = '/mnt/e/rms/acc_csazores/'

ipk = 'bgri'

epsg = 5015

In [ ]:
iaf = lst_fld(folder)

for f in iaf:
    cf_ign_distfac(
        os.path.join(f, 'incidents.shp'), ipk,
        os.path.join(f, 'facilities.shp'),
        os.path.join(f, 'cf_result_v2.shp'),
        crs=epsg, save_temp_json=True
    )

In [ ]:
iaf

-------------------------------------

In [ ]:
from glass.mob.esri.cf import cf_based_on_relations

i = '/mnt/e/rms/azores/rms_bgri21_pnt.shp'
ipk = 'bgri'
igroup = 'idfreg'

f = '/mnt/e/rms/equipasaude/cs_azores.shp'
fpk = 'csid'

reltbl = '/mnt/e/rms/equipasaude/AInf_CSP_Azores.xlsx'
sheet = 'A_Inf'
groupfk = 'ddccff'
ffk = 'idcs'

epsg = 5015

output = '/mnt/e/rms/acc_pop21/azores_cs.shp'

cf_based_on_relations(
    i, ipk, igroup,
    f, fpk, reltbl, sheet, groupfk, ffk,
    output, impedance='TravelTime',
    srs=epsg
)

In [ ]:
from glass.cons.esri import TV_URL, CF_URL, rest_token

from glass.rd.shp import shp_to_obj
from glass.prj.obj import df_prj
from glass.prop.prj import get_epsg
import requests as rq
from glass.cons.esri import get_tv_by_impedancetype
import json as js
from glass.it.esri    import json_to_gjson

In [ ]:
i   = '/mnt/d/rms/acc_pop21/tst_incidents.shp'
ipk = 'bgri'

f = '/mnt/d/rms/acc_pop21/tst_facilities.shp'
out = '/mnt/d/rms/acc_pop21/azores_idosos.shp'

In [ ]:
tv = get_tv_by_impedancetype("WalkTime")

In [ ]:
token = rest_token()

In [ ]:
tvs = rq.get(TV_URL, params={'f' : 'json', 'token' : token})

In [ ]:
tvs.json()

In [ ]:
epsg

In [ ]:
idf = shp_to_obj(i, srs_to=4326 if epsg != 4326 else None)
idf['lat'] = idf.geometry.y.astype(str)
idf['lng'] = idf.geometry.x.astype(str)

fdf = shp_to_obj(f, srs_to=4326 if epsg != 4326 else None)
fdf['lat'] = fdf.geometry.y.astype(str)
fdf['lng'] = fdf.geometry.x.astype(str)

idf["coords"] = idf.lng + ',' + idf.lat
fdf["coords"] = fdf.lng + ',' + fdf.lat

istr = idf.coords.str.cat(sep=";")
fstr = fdf.coords.str.cat(sep=";")

In [ ]:
fstr

In [ ]:
r = rq.get(CF_URL, params={
    #"impedanceAttributeName" : "TravelTime"
    'facilities'             : fstr,
    'incidents'              : istr,
    'token'                  : token,
    'f'                      : 'json',
    'travelModel'            : js.dumps(tv),
    'defaultTargetFacilityCount' : '1',
    'returnCFRoutes'         : True,
    'travelDirection'        : 'esriNATravelDirectionToFacility',
    'impedanceAttributeName' : 'WalkTime'
})

In [ ]:
r.status_code

In [ ]:
esri_geom = r.json()
geom = json_to_gjson(esri_geom.get('routes'))

In [ ]:
geom

In [ ]:
r.json()

In [ ]:
tv={
    "attributeParameterValues": [{
        "parameterName": "Restriction Usage",
        "attributeName": "Walking",
        "value": "PROHIBITED"
    },{
        "parameterName": "Restriction Usage",
        "attributeName": "Preferred for Pedestrians","value": "PREFER_LOW"},{"parameterName": "Walking Speed (km/h)","attributeName": "WalkTime","value": 5}],"description": "Follows paths and roads that allow pedestrian traffic and finds solutions that optimize travel time. The walking speed is set to 5 kilometers per hour.","impedanceAttributeName": "WalkTime","simplificationToleranceUnits": "esriMeters","uturnAtJunctions": "esriNFSBAllowBacktrack","restrictionAttributeNames": ["Preferred for Pedestrians","Walking"],"useHierarchy": false,"simplificationTolerance": 2,"timeAttributeName": "WalkTime","distanceAttributeName": "Miles","type": "WALK","id": "caFAgoThrvUpkFBW","name": "Walking Time"}

In [ ]:
tv